In [3]:
!pip install plotly
!pip install scikit-learn
!pip install numpy
!pip install pandas
!pip install seaborn

ERROR: Operation cancelled by user
^C


In [2]:
import sklearn as sk
import numpy as np
import plotly.express as px
import pandas as pd
import seaborn as sns

In [4]:
# Loading Datasets
data = pd.read_csv('./data.csv', usecols=['DR_NO','AREA NAME' ,'Date Rptd', 'DATE OCC', 'TIME OCC', 'LOCATION', 'Crm Cd Desc', 'Vict Age', 'Vict Sex', 'Weapon Desc', 'Premis Desc', 'LAT', 'LON'], header=0)
data = data[data['DATE OCC'].str.contains('2024')]
data.dropna(inplace=True)



encoded = pd.DataFrame()

data.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA NAME,Crm Cd Desc,Vict Age,Vict Sex,Premis Desc,Weapon Desc,LOCATION,LAT,LON
0,240404866,01/29/2024 12:00:00 AM,2024-01-28,1735,Hollenbeck,CRIMINAL HOMICIDE,14,M,STREET,HAND GUN,1ST,34.0450,-118.2256
1,242108414,04/25/2024 12:00:00 AM,2024-04-25,1600,Topanga,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),0,M,STREET,HAND GUN,21400 SHERMAN WY,34.2010,-118.5977
2,240604934,01/21/2024 12:00:00 AM,2024-01-21,1510,Hollywood,BATTERY - SIMPLE ASSAULT,22,F,MISSIONS/SHELTERS,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",1300 N WESTERN AV,34.0944,-118.3125
3,242107187,03/22/2024 12:00:00 AM,2024-03-22,1815,Topanga,BATTERY - SIMPLE ASSAULT,58,M,SIDEWALK,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",22000 GILMORE ST,34.1876,-118.6070
4,240310004,04/25/2024 12:00:00 AM,2024-04-25,1700,Southwest,BATTERY - SIMPLE ASSAULT,19,F,PARKING LOT,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",3000 S FIGUEROA ST,34.0268,-118.2808


In [ ]:
gender_counts = data['Vict Sex'].value_counts()
gender_counts

In [ ]:
data.describe()

In [ ]:
# Time Conversion

# Combine DATE OCC and TIME OCC into a single datetime column
data['DATE OCC'] = pd.to_datetime(data['DATE OCC'], format='%m/%d/%Y %I:%M:%S %p')
#data['DATE OCC'] = pd.to_datetime(data['DATE OCC'] + ' ' + data['TIME OCC'], format='%Y-%m-%d %M:%S')
# Convert the datetime column to Unix timestamps
encoded['UNIX_TIMESTAMP'] = data['DATE OCC'].apply(lambda x: x.timestamp())
encoded['TIME OCC'] = data['TIME OCC'].copy()
data

# Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit the LabelEncoder to the 'Crm Cd Desc' column
label_encoder.fit(data['Crm Cd Desc'])
# Transform the 'Crm Cd Desc' column
encoded['Crm Cd Desc'] = label_encoder.transform(data['Crm Cd Desc'])

# Fit the LabelEncoder to the 'AREA NAME' column
label_encoder.fit(data['AREA NAME'])
# Transform the 'AREA NAME' column
encoded['AREA NAME'] = label_encoder.transform(data['AREA NAME'])

# Fit the LabelEncoder to the 'AREA NAME' column
label_encoder.fit(data['Vict Sex'])
# Transform the 'AREA NAME' column
encoded['Vict Sex'] = label_encoder.transform(data['Vict Sex'])

# Fit the LabelEncoder to the 'AREA NAME' column
label_encoder.fit(data['Premis Desc'])
# Transform the 'AREA NAME' column
encoded['Premis Desc'] = label_encoder.transform(data['Premis Desc'])

# Fit the LabelEncoder to the 'AREA NAME' column
label_encoder.fit(data['Weapon Desc'])
# Transform the 'AREA NAME' column
encoded['Weapon Desc'] = label_encoder.transform(data['Weapon Desc'])

# Fit the LabelEncoder to the 'AREA NAME' column
label_encoder.fit(data['LOCATION'])
# Transform the 'AREA NAME' column
encoded['LOCATION'] = label_encoder.transform(data['LOCATION'])


encoded.to_csv('encoded.csv', index=True)
data.to_csv('data.csv', index=True)
encoded


In [ ]:
import matplotlib.pyplot as plt

# Assuming 'encoded' is your DataFrame with encoded data
# Compute the correlation matrix
correlation_matrix = encoded.corr()

# Display the correlation matrix
print(correlation_matrix)

# Optionally, visualize the correlation matrix using a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()